In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks

In [ ]:
import shutil
file_name = "./data/creditcard.csv.zip"
output_dir = "./data"
format = "zip"
shutil.unpack_archive(file_name, output_dir, format)

In [ ]:
import numpy as np
import pandas as pd
import sklearn
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.model_selection  import cross_val_score
from imblearn.over_sampling import SMOTE
from sklearn.metrics import precision_score, recall_score, f1_score, mean_squared_error
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from sklearn.cluster import KMeans

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout

import lightgbm as lgb


import warnings
warnings.filterwarnings("ignore")


In [ ]:
df = pd.read_csv('./data/creditcard.csv',sep=',')

In [ ]:
y = df['Class']
X = df.drop(["Class",'Time'], axis=1)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42,test_size=0.3)


In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)

X_test_scaled = scaler.transform(X_test)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_train.columns, index=X_test.index)

In [ ]:
X_majority = X_train_scaled_df[y_train == 0]
y_majority = y_train[y_train == 0]

# 2. k-means 클러스터링
n_clusters = 15  # 클러스터 개수를 15로 설정합니다.
kmeans = KMeans(n_clusters=n_clusters, random_state=42).fit(X_majority)
clusters = kmeans.predict(X_majority)


cluster_data_list = [X_majority[clusters == i] for i in range(n_clusters)]
cluster_sizes = [len(cluster_data) for cluster_data in cluster_data_list]

# 3. 여러 개의 랜덤 서브셋 생성
num_subsets = 10  # 생성할 서브셋의 개수
total_sample_size = 20000  # 총 샘플 수
all_sampled_subsets = []

def sample_without_replacement(cluster_data, n_samples, used_indices):
    available_indices = list(set(range(len(cluster_data))) - used_indices)
    if len(available_indices) < n_samples:
        n_samples = len(available_indices)
    sampled_indices = np.random.choice(available_indices, size=n_samples, replace=False)
    used_indices.update(sampled_indices)
    return cluster_data.iloc[sampled_indices]

for _ in range(num_subsets):
    remaining_samples = total_sample_size
    final_sampled_subset = pd.DataFrame()
    used_indices_list = [set() for _ in range(n_clusters)]

    while remaining_samples > 0:
        for i, cluster_data in enumerate(cluster_data_list):
            if remaining_samples <= 0:
                break
            available_samples = len(cluster_data) - len(used_indices_list[i])
            if available_samples > 0:
                if cluster_sizes[i] < 1300:
                    n_samples = available_samples
                else:
                    n_samples = min(1300, available_samples, remaining_samples)
                sampled_data = sample_without_replacement(cluster_data, n_samples, used_indices_list[i])
                final_sampled_subset = pd.concat([final_sampled_subset, sampled_data])
                remaining_samples -= n_samples
            if len(final_sampled_subset) >= total_sample_size:
                final_sampled_subset = final_sampled_subset.iloc[:total_sample_size]
                remaining_samples = 0
                break
    all_sampled_subsets.append(final_sampled_subset)


In [ ]:
from numpy.linalg import slogdet, inv
import numpy as np

def kl_divergence(mu_p, sigma_p, mu_q, sigma_q):
    """
    Calculate the KL divergence between two multivariate Gaussian distributions.

    Parameters:
    mu_p (np.ndarray): Mean vector of the distribution p
    sigma_p (np.ndarray): Covariance matrix of the distribution p
    mu_q (np.ndarray): Mean vector of the distribution q
    sigma_q (np.ndarray): Covariance matrix of the distribution q

    Returns:
    float: The KL divergence D_KL(p || q)
    """
    D = mu_p.shape[0]  # Dimension of the distributions

    # Calculate the determinant and inverse of sigma_q
    det_sigma_p = np.linalg.det(sigma_p)
    det_sigma_q = np.linalg.det(sigma_q)
    inv_sigma_q = np.linalg.inv(sigma_q)

    # Calculate the first term: 0.5 * log(det_sigma_q / det_sigma_p)
    term1 = 0.5 * np.log(det_sigma_q / det_sigma_p) # 두 분포의 공분산 행렬의 행렬식 비율의 로그. 공분산 행렬의 행렬식은 분포의 "확산 정도"를 나타내므로, 두 분포의 확산 정도 차이를 로그 비율

    # Calculate the second term: -D/2
    term2 = -0.5 * D # 분포의 차원 D에 대한 조정 항, KL 발산 값이 분포의 차원에 대해 조정

    # Calculate the third term: 0.5 * trace(inv_sigma_q @ sigma_p)
    term3 = 0.5 * np.trace(inv_sigma_q @ sigma_p) # 분포 q의 공분산 행렬의 역행렬과 분포 p의 공분산 행렬의 곱의 대각합 공분산 행렬의 곱의 대각합은 두 분포의 상관 관계와 분산의 차이

    # Calculate the fourth term: 0.5 * (mu_p - mu_q).T @ inv_sigma_q @ (mu_p - mu_q)
    diff_mu = mu_p - mu_q
    term4 = 0.5 * diff_mu.T @ inv_sigma_q @ diff_mu # 두 분포의 평균 벡터 차이에 대한 조정 항, @는 행렬 곱셈 연산자, 이 항은 두 분포의 중심(평균 벡터)의 차이를 측정합니다. 두 분포의 평균 벡터가 얼마나 다른지.

    # Sum all the terms to get the KL divergence
    kl_div = term1 + term2 + term3 + term4

    return kl_div

kl_divergences = []
mu_p = X_majority.mean().values
sigma_p = np.cov(X_majority, rowvar=False) # 원본 데이터의 공분산 행렬

for subset in all_sampled_subsets:
    mu_q = subset.mean().values
    sigma_q = np.cov(subset, rowvar=False) # Subest의 공분산 행렬
    kl_divergences.append(kl_divergence(mu_p, sigma_p, mu_q, sigma_q)) # 원본 데이터와 subset의 mu(모집단 평균)와 sigma(공분산 행렬) 비교

# 5. 최적 subset 선택
best_subset_idx = np.argmin(kl_divergences)
best_subset = all_sampled_subsets[best_subset_idx]

X_minority = X_train_scaled_df[y_train == 1]
y_minority = y_train[y_train == 1]

X_balanced = pd.concat([X_minority, best_subset])
y_balanced = pd.concat([y_minority, pd.Series([0]*len(best_subset))])


In [ ]:
def evaluate(model):
  model.fit(X_train,y_train)
  pre = model.predict(X_test)

  accuracy = accuracy_score(pre,y_test)
  precision = precision_score(pre,y_test)
  recall = recall_score(pre,y_test)
  f1 = f1_score(pre,y_test)

  sns.heatmap(confusion_matrix(pre,y_test),annot=True)
  print(model)
  print('Accuracy : ',accuracy,'Recall : ',recall, "Precision : ",precision,"F1 : ",f1)


In [ ]:
clf = lgb.LGBMClassifier(random_state=42)
clf.fit(X_train, y_train)

evaluate(clf)

In [ ]:
clf = lgb.LGBMClassifier(random_state=42)
clf.fit(X_balanced, y_balanced)
pre = clf.predict(X_test)

accuracy = accuracy_score(pre,y_test)
precision = precision_score(pre,y_test)
recall = recall_score(pre,y_test)
f1 = f1_score(pre,y_test)

sns.heatmap(confusion_matrix(pre,y_test),annot=True)
print(clf)
print('Accuracy : ',accuracy,'Recall : ',recall, "Precision : ",precision,"F1 : ",f1)